# ONS Core

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os
import csv

import common_core

## Configuration

Data to download from the NHS statistical work area

In [2]:
# Only using England for now due to the availability of age demographics, etc
nationNames = ["England"]

# All 7 NHS regions in England
nhsRegionNames = ["North West", "North East and Yorkshire", "Midlands",
                  "East of England", "London", "South East", "South West"]

# Combine all of these area types into a single list
areas = [("nation", ["England"]), ("nhsregion", nhsRegionNames)]

In [3]:
skipExisting = True

deathsUrl = "https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales"
deathsPath = os.path.join(common_core.projdir, "data", "ons-deaths", "raw")

deathsFiles = [
    ("weekly", ".*\.xls$"),
    ("weekly", ".*\.xlsx$")
]

In [4]:
ageDemographics = [
    '00_04', '05_09', '10_14', '15_19', '20_24', '25_29',
    '30_34', '35_39', '40_44', '45_49', '50_54', '55_59',
    '60_64', '65_69', '70_74', '75_79', '80_84', '85_89', '90+'
]

## Download Data

Download spreadsheets by parsing the HTML for suitable links

In [5]:
def downloadDeaths():
    webDownload = common_core.WebDownload(skipExisting=skipExisting)
    files = webDownload.downloadFiles(deathsPath, deathsUrl, deathsFiles)